In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [7]:
df.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [6]:
# Handle missing values (e.g., fill with mean, median, or mode)
df['genre'].fillna(df['genre'].mode()[0], inplace=True)
df['type'].fillna(df['type'].mode()[0],inplace=True)   
df['rating'].fillna(df['rating'].mean(), inplace=True)


In [11]:
# Display the first few rows of the dataset
print(df.head())

# Get summary statistics of the dataset
print(df.describe())

# Explore unique values in categorical features (e.g., genre, type)
print("Unique Genres:", df['genre'].unique())
print("Unique Types:", df['type'].unique())


   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  
           anime_id        rating       members
count  12294.000000  12294.000000  1.229400e+04
mean   14058.221653      6.473902  1.807134e+04
st

In [12]:
# Features: genres, type, rating
features = df[['genre', 'type', 'rating']]


In [13]:
# Convert categorical features into numerical representations using one-hot encoding
features = pd.get_dummies(features)

In [14]:
# Normalize numerical features (e.g., rating)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features['rating'] = scaler.fit_transform(features[['rating']])


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_anime(target_anime, features, threshold=0.5):
    # Compute cosine similarity
    cosine_sim = cosine_similarity(features)
    
    # Get index of target anime
    index = df[df['name'] == target_anime].index[0]
    
    # Get similarity scores for target anime
    sim_scores = list(enumerate(cosine_sim[index]))
    
    # Sort scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get similar anime with similarity scores above threshold
    similar_anime = [(df.iloc[i]['name'], score) for i, score in sim_scores if score > threshold]
    
    return similar_anime


In [18]:
target_anime = 'Kimi no Na wa.'  # Example target anime
recommended_anime = recommend_anime(target_anime, features)
print("Recommended Anime for '{}':".format(target_anime))
for anime, score in recommended_anime:
    print(anime, ":", score)


Recommended Anime for 'Kimi no Na wa.':
Kimi no Na wa. : 1.0
Taka no Tsume 8: Yoshida-kun no X-Files : 0.6576062862691243
Mogura no Motoro : 0.6514132938870387
Kahei no Umi : 0.6491235103542037
Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare : 0.6458697194244033
Koe no Katachi : 0.6451381167094017
Mirai ni Mukete: Bousai wo Kangaeru : 0.6443977093165866
Okaachan Gomen ne : 0.6443977093165866
Shaka no Shougai : 0.6443977093165866
Shenmi Shijie Lixian Ji : 0.6443977093165866
Sen to Chihiro no Kamikakushi : 0.6433462248353273
Ookami Kodomo no Ame to Yuki : 0.6419684906254086
Mononoke Hime : 0.6415027311252344
Suzumiya Haruhi no Shoushitsu : 0.6415027311252344
Mushishi Zoku Shou: Suzu no Shizuku : 0.64056135722548
Maya no Isshou : 0.64056135722548
Howl no Ugoku Shiro : 0.6404031781405257
Kizumonogatari II: Nekketsu-hen : 0.6402446310019909
Kara no Kyoukai 5: Mujun Rasen : 0.6394463567115418
Shisei Sasshin : 0.6392855905787217
Tengen Toppa Gurren Lagann Movie: Lagann-hen : 0.63880106050

In [19]:
# Example: Adjust threshold to 0.7
recommended_anime = recommend_anime(target_anime, features, threshold=0.7)
print("Recommended Anime for '{}' (Threshold=0.7):".format(target_anime))
for anime, score in recommended_anime:
    print(anime, ":", score)


Recommended Anime for 'Kimi no Na wa.' (Threshold=0.7):
Kimi no Na wa. : 1.0
